In [8]:
import numpy as np
import pandas as pd
from scipy.stats import norm as normal, binom
import bplot as bp
from scipy.optimize import minimize
from scipy.special import loggamma
import patsy
import multiprocessing
from joblib import Parallel, delayed

In [27]:
def ll_normal(beta, yX):
    y = yX[:, 0]
    X = yX[:, 1:]
    N = X.shape[0]
    mu = np.full(N, np.nan)
    for n in range(N):
        mu[n] = np.sum(X[n, :] * beta)
        
    d = y - mu
    return np.sum(d*d)

def bootstrap(data, R, fun, initval = None):
    N=data.shape[0]
    k=data.shape[1]
    k -= 1
    thetas = np.full((R, k), np.nan)
    for r in range(R):
        idx = np.random.choice(N, N, replace=True)
        thetas[r, :] = fun(data[idx, :], initval)
    return thetas
        
def optim(data, initval = None):
    k = data.shape[1]-1
    return minimize(ll_normal, (initval if a.all(initval) else np.random.normal(size = k)), args=(data), method="BFGS")["x"]

def boot(data, N, fun, initval):
    idx = np.random.choice(N, N)
    return fun(data[idx, :], initval)

def pbootstrap(data, R, fun, ncpus=multiprocessing.cpu_count(), initval = None):
    N=data.shape[0]
    thetas = Parallel(ncpus)(delayed(boot)(data, N, fun, initval) for _ in range(R))
    return np.asarray(thetas)

# 15 November 2019

In [16]:
df = pd.read_csv("https://raw.githubusercontent.com/roualdes/data/master/elmhurst.csv")
X = patsy.dmatrix("~ family_income", data=df)
yX = np.c_[df["gift_aid"], X]

In [28]:
betahat = optim(yX)
betas = pbootstrap(yX, 1001, optim, betahat)

NameError: name 'a' is not defined

In [ ]:
optim(yX)